In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize , pos_tag, sent_tokenize
import warnings
from gensim import corpora , models , similarities
import jieba
import re

## Underthesea - tao dataframe

In [174]:
data_product = pd.read_excel('data_news_3.xlsx')  
data_product.tail(3)

,Unnamed: 0,link,content,description
144,144,https://vnexpress.net/loi-the-khi-tham-kham-ma...,"[<p class=""Normal"">Công nghệ phát triển, những...",Phòng khám FSEC được thiết kế với nhiều màu sắ...
145,145,https://vnexpress.net/du-bao-xu-huong-cua-thi-...,"[<p class=""Normal"">Ông Dương Ngọc Báu, CEO Côn...","Dù khó khăn vẫn còn tồn tại, các doanh nghiệp ..."
146,146,https://vnexpress.net/bleu-beaute-dat-muc-tieu...,"[<p class=""Normal"">Bleu Beaute Skincare Clinic...","Theo Bleu Beaute, đơn vị này mong muốn cung cấ..."


In [175]:
# data_review = pd.read_csv('Review.csv')

In [176]:
# change data type of review_id and product_id to int
# data_review = data_review.astype(int)

In [177]:
# data_review.tail(3)

In [178]:
# data_product.loc[data_product['item_id'] == 9996258]

##### Cot item_id ung voi cot product_id

#### tao data, su dung gensim de phan tich cot 'name'

In [179]:
data_product = data_product['description'].to_frame()
# data_product.tail(2)
# change name column

data_product.columns = ['product_name']

In [180]:
data_product.head(2)

,product_name
0,"Mỗi lít xăng giảm 370-500 đồng, các mặt hàng d..."
1,"Tiến sĩ, Chuyên gia Nghiên cứu và Phát triển, ..."


In [181]:
# drop na
data_product.dropna(inplace=True)

In [182]:
# work_tokenize 

data_product['product_name_wt'] = data_product['product_name'].apply(lambda x: word_tokenize(x, format="text"))

In [183]:
data_product.tail(2)

,product_name,product_name_wt
145,"Dù khó khăn vẫn còn tồn tại, các doanh nghiệp ...","Dù khó_khăn vẫn còn tồn_tại , các doanh_nghiệp..."
146,"Theo Bleu Beaute, đơn vị này mong muốn cung cấ...","Theo Bleu_Beaute , đơn_vị này mong_muốn cung_c..."


In [184]:
# pos_tag

product_pos_tag = data_product['product_name'].apply(lambda x: pos_tag(x, format="text"))
product_pos_tag

0      [(Mỗi, M), (lít, Nu), (xăng, N), (giảm, V), (3...
1      [(Tiến sĩ, N), (,, CH), (Chuyên gia, N), (Nghi...
2      [(Bộ, N), (Văn hóa, N), (Thể thao, N), (và, C)...
4      [(Kho, N), (lưu trữ, N), (của, E), (Viện, N), ...
5      [(Theo, V), (cựu, A), (danh thủ, N), (Jorge Va...
                             ...                        
142    [(Tiếp nối, V), (hành trình, N), (", CH), (Mob...
143    [(Thương, V), (hiệu, N), (sơn, N), (thuộc, V),...
144    [(Phòng khám, N), (FSEC, Np), (được, V), (thiế...
145    [(Dù, C), (khó khăn, A), (vẫn, R), (còn, R), (...
146    [(Theo, V), (Bleu Beaute, Np), (,, CH), (đơn v...
Name: product_name, Length: 145, dtype: object

In [185]:
# sent_tokenize 
product_sent = data_product['product_name'].apply(lambda x: sent_tokenize(x))
product_sent

0      [Mỗi lít xăng giảm 370-500 đồng, các mặt hàng ...
1      [Tiến sĩ, Chuyên gia Nghiên cứu và Phát triển,...
2      [Bộ Văn hóa Thể thao và Du lịch đề xuất Chính ...
4      [Kho lưu trữ của Viện Nghiên cứu Hán Nôm mất 2...
5      [Theo cựu danh thủ Jorge Valdano, Lionel Messi...
                             ...                        
142    [Tiếp nối hành trình "MobiFone chia sẻ", chươn...
143    [Thương hiệu sơn thuộc AkzoNobel ứng dụng công...
144    [Phòng khám FSEC được thiết kế với nhiều màu s...
145    [Dù khó khăn vẫn còn tồn tại, các doanh nghiệp...
146    [Theo Bleu Beaute, đơn vị này mong muốn cung c...
Name: product_name, Length: 145, dtype: object

#### Gensim

In [186]:
# tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in data_product.product_name_wt]

In [187]:
products_gem[:1]

[['Mỗi',
  'lít',
  'xăng',
  'giảm',
  '370',
  '-',
  '500',
  'đồng',
  ',',
  'các',
  'mặt_hàng',
  'dầu',
  'cũng',
  'hạ',
  '70-150',
  'đồng',
  'một',
  'lít',
  ',',
  'kg',
  'tùy',
  'loại',
  ',',
  'từ',
  '15',
  'h',
  'hôm_nay',
  '.']]

In [188]:
# using regular expression to remove special characters
import re

with open('vietnamese-stopwords.txt' , 'r', encoding='utf8') as stop_words:
    stop_words = stop_words.read().split('\n')

In [189]:
stop_words[:5]

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy']

In [190]:
# remove some special characters
def ham_xu_ly(stop_words , products_gem):
    products_gem_re = [[re.sub('[0-9]+' , '', e) for e in text] for text in products_gem] # so
    products_gem_re = [[t.lower() for t in text if not t in['', ' ', ',', '.', '...', '-',':',';','?', '%','(', ')','+', '/', 'g','ml']]for text in products_gem_re] # ky tu dac biet
    products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re]  # stopword
    return products_gem_re

In [191]:
products_gem_re = ham_xu_ly(stop_words , products_gem)

In [192]:
# using gensim to create dictionary
dictionary = corpora.Dictionary(products_gem_re)

In [193]:
# list of words in the dictionary
dictionary.token2id

{'dầu': 0,
 'h': 1,
 'hôm_nay': 2,
 'hạ': 3,
 'kg': 4,
 'lít': 5,
 'mặt_hàng': 6,
 'tùy': 7,
 'xăng': 8,
 'đồng': 9,
 'chuyên_gia': 10,
 'covid-': 11,
 'gia_đình': 12,
 'muộn': 13,
 'mất_mát': 14,
 'nghiên_cứu': 15,
 'người_thân': 16,
 'phát_triển': 17,
 'thăm': 18,
 'tiến_sĩ': 19,
 'trung_tuần': 20,
 'trở_về': 21,
 'việt_nam': 22,
 'đêm': 23,
 'đại_dịch': 24,
 'chính_phủ': 25,
 'cửa_khẩu': 26,
 'kéo_dài': 27,
 'lưu_trú': 28,
 'quốc_tế': 29,
 'thí_điểm': 30,
 'thể_thao': 31,
 'thị_thực_tại': 32,
 'văn_hóa': 33,
 'xem_xét': 34,
 'đề_xuất': 35,
 '"': 36,
 'cổ': 37,
 'hán': 38,
 'kho': 39,
 'lê_quý_đôn': 40,
 'lưu_trữ': 41,
 'lục': 42,
 'nôm': 43,
 'sách': 44,
 'thi': 45,
 'toàn': 46,
 'viện': 47,
 'việt': 48,
 'argentina': 49,
 'bảo_vệ': 50,
 'chức': 51,
 'cựu': 52,
 'danh_thủ': 53,
 'hứa': 54,
 'jorge_valdano': 55,
 'lionel_messi': 56,
 'phỏng_vấn': 57,
 'vô_địch': 58,
 'đội_tuyển': 59,
 'cầu': 60,
 'giao_thông': 61,
 'giám_đốc': 62,
 'hà_nội': 63,
 'lo': 64,
 'lân_cận': 65,
 'ngã_tư_sở

In [194]:
len(dictionary)

1074

In [195]:
# feature_cnt
feature_cnt = len(dictionary.token2id.keys())

In [196]:
feature_cnt

1074

In [197]:
# get corpus
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [198]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2)]

In [199]:
# use tfidf model to transform the corpus
tfidf = models.TfidfModel(corpus)
#  sparse matrix
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)

In [231]:
# select the second product as the query

product_selection = data_product.iloc[2]

# convert to dataframe

product_selection = pd.DataFrame(product_selection).T

In [232]:
product_selection

,product_name,product_name_wt
2,Bộ Văn hóa Thể thao và Du lịch đề xuất Chính p...,Bộ Văn_hóa Thể_thao và Du_lịch đề_xuất Chính_p...


In [233]:
type(product_selection)

pandas.core.frame.DataFrame

In [234]:
# get the product name
name_description_pre = product_selection['product_name_wt'].to_string(index=False)
name_description_pre

'Bộ Văn_hóa Thể_thao và Du_lịch đề_xuất Chính_ph...'

In [235]:
view_product = name_description_pre.lower().split()
view_product

['bộ', 'văn_hóa', 'thể_thao', 'và', 'du_lịch', 'đề_xuất', 'chính_ph...']

In [236]:
# convert search product to vector
kw_vector = dictionary.doc2bow(view_product)

In [237]:
kw_vector

[(31, 1), (33, 1), (35, 1)]

In [238]:
# similarity calculation
sims = index[tfidf[kw_vector]]

In [239]:
# sort the similarity
sims = sorted(enumerate(sims), key=lambda item: -item[1])

# get the most similar products
sims[:10]

[(2, 0.5025902),
 (129, 0.12072664),
 (14, 0.09483387),
 (52, 0.09483387),
 (0, 0.0),
 (1, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0)]

In [240]:
# get name of the most similar products

def get_name_product(sims):
    name_product = []
    for i in sims:
        name_product.append(data_product['product_name'].iloc[i[0]])
    return name_product

In [241]:
get_name_product(sims[:3])

['Bộ Văn hóa Thể thao và Du lịch đề xuất Chính phủ kéo dài thời gian lưu trú với khách quốc tế lên 30 ngày và xem xét thí điểm cấp thị thực tại cửa khẩu.',
 'Doanh nghiệp nên tập trung vào con người và văn hóa công ty trong thời kỳ suy thoái kinh tế để đảm bảo tính gắn bó với nhân sự, theo các chuyên gia.',
 'Sở hữu lợi thế về thiên nhiên, văn hoá... vượt trội so với nước lân cận nhưng khách quốc tế không trở lại Việt Nam vốn là "nỗi đau" từ nhiều năm, chính sách bất nhất sau Covid là đỉnh điểm khiến du lịch thất bại, theo chuyên gia.']

In [166]:
# # crawl news from vnexpress

# import requests
# from bs4 import BeautifulSoup

# url = 'https://vnexpress.net/'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')

# # get title of news
# title_news = soup.find_all('h3', class_ = 'title-news')
# title_news

# title_news_list = [i.text for i in title_news]
# title_news_list

# # get link of news
# link_news = soup.find_all('h3', class_ = 'title-news')
# link_news

# link_news_list = [i.find('a')['href'] for i in link_news]
# link_news_list

# # get content of news
# content_news_list = []

# for i in link_news_list:
#     response = requests.get(i)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     content_news = soup.find_all('p', class_ = 'Normal')
#     content_news_list.append(content_news)

# content_news_list

# # get description of news

# description_news_list = []

# for i in link_news_list:
#     response = requests.get(i)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     description_news = soup.find_all('p', class_ = 'description')
#     description_news_list.append(description_news)

# len(description_news_list)



# # create dataframe

# import pandas as pd

# data_news = pd.DataFrame({
#     'title': title_news_list,
#     'link': link_news_list,
#     'content': content_news_list,
#     'description': description_news_list
# })

# data_news

# # save to excel

# # data_news.to_excel('data_news.xlsx')

In [45]:
# # delete <p class="description"> in description

# data_news['description'] = data_news['description'].astype(str)
# data_news['description'] = data_news['description'].str.replace('<p class="description">', '')
# data_news['description'] = data_news['description'].str.replace('</p>', '')
# data_news['description'] = data_news['description'].str.replace('[', '')
# data_news['description'] = data_news['description'].str.replace(']', '')

# data_news.tail(2)

# # data_news.to_excel('data_news_1.xlsx')

In [ ]:


# # split description 

# data_news['description'] = data_news['description'].apply(lambda x: x.split('>'))



In [46]:
# # get the last element of description

# data_news['description'] = data_news['description'].apply(lambda x: x[-1])



In [ ]:
# delete '' element in description

# data_news.to_excel('data_news_2.xlsx')

In [ ]:
# crawl news from vnexpress

import requests
from bs4 import BeautifulSoup

url = 'https://vnexpress.net/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# get title of news
title_news = soup.find_all('h3', class_ = 'title-news')
title_news

title_news_list = [i.text for i in title_news]
title_news_list

# get link of news
link_news = soup.find_all('h3', class_ = 'title-news')
link_news

link_news_list = [i.find('a')['href'] for i in link_news]
link_news_list

# get content of news
content_news_list = []

for i in link_news_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    content_news = soup.find_all('p', class_ = 'Normal')
    content_news_list.append(content_news)

content_news_list

# get description of news

description_news_list = []

for i in link_news_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    description_news = soup.find_all('p', class_ = 'description')
    description_news_list.append(description_news)

len(description_news_list)



# create dataframe

import pandas as pd

data_news = pd.DataFrame({
    'title': title_news_list,
    'link': link_news_list,
    'content': content_news_list,
    'description': description_news_list
})

data_news

# save to excel

# data_news.to_excel('data_news.xlsx'

In [ ]:
# crawl news from vnexpress automaticly by selenium

from selenium import webdriver
from bs4 import BeautifulSoup
import time

path = 'D:\WORK&LEARN\TRAN MINH VU\VISUAL_STUDIO_CODE/fbchat\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome( path)

url = 'https://vnexpress.net/'
driver.get(url)
time.sleep(5)

# scroll down to load more news
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

# get title of news
soup = BeautifulSoup(driver.page_source, 'html.parser')
title_news = soup.find_all('h3', class_ = 'title-news')
title_news

title_news_list = [i.text for i in title_news]
title_news_list

# get link of news
link_news = soup.find_all('h3', class_ = 'title-news')
link_news

link_news_list = [i.find('a')['href'] for i in link_news]
link_news_list

# # get content of news
# content_news_list = []

# for i in link_news_list:
#     response = requests.get(i)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     content_news = soup.find_all('p', class_ = 'Normal')
#     content_news_list.append(content_news)

# content_news_list

# # get description of news

# description_news_list = []

# for i in link_news_list:
#     response = requests.get(i)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     description_news = soup.find_all('p', class_ = 'description')
#     description_news_list.append(description_news)

# len(description_news_list)

In [102]:
len(link_news_list)

162

In [120]:
# delete some error links
k = 0
link_news_list_2 = []
for i in link_news_list:
    z = re.match('https://vnexpress.net/', i)
    if z:
        k = k + 1
        link_news_list_2.append(i)

len(link_news_list_2)
        

147

In [121]:
# get content of news
content_news_list = []

for i in link_news_list_2:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    content_news = soup.find_all('p', class_ = 'Normal')
    content_news_list.append(content_news)

content_news_list

# get description of news

description_news_list = []

for i in link_news_list_2:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    description_news = soup.find_all('p', class_ = 'description')
    description_news_list.append(description_news)



In [123]:
data_news = pd.DataFrame({
    # 'title': title_news_list,
    'link': link_news_list_2,
    'content': content_news_list,
    'description': description_news_list
})

data_news

,link,content,description
0,https://vnexpress.net/gia-xang-dau-giam-tiep-v...,[[Theo điều chỉnh của liên Bộ Công Thương - Tà...,"[[Mỗi lít xăng giảm 370-500 đồng, các mặt hàng..."
1,https://vnexpress.net/mot-di-khong-tro-lai-455...,[[Niềm xúc động khi được đặt chân đến Tân Sơn ...,"[[Tiến sĩ, Chuyên gia Nghiên cứu và Phát triển..."
2,https://vnexpress.net/de-xuat-thoi-gian-luu-tr...,"[[Sáng 21/12, Thủ tướng Phạm Minh Chính chủ tr...",[[Bộ Văn hóa Thể thao và Du lịch đề xuất Chính...
3,https://vnexpress.net/vi-sao-tran-argentina-ph...,[],[]
4,https://vnexpress.net/vien-nghien-cuu-han-nom-...,"[[Sáng 21/12, Tiến sĩ Đào Phương Chi - trưởng ...","[[[Hà Nội], Kho lưu trữ của Viện Nghiên cứu Há..."
...,...,...,...
142,https://vnexpress.net/hoi-cho-que-mobifone-tie...,[[Sau nhiều chương trình hướng tới cộng đồng n...,"[[Tiếp nối hành trình ""MobiFone chia sẻ"", chươ..."
143,https://vnexpress.net/dulux-dung-ai-phac-hoa-k...,"[[Hoạt động nằm trong khuôn khổ chiến dịch ""Tậ...",[[Thương hiệu sơn thuộc AkzoNobel ứng dụng côn...
144,https://vnexpress.net/loi-the-khi-tham-kham-ma...,"[[Công nghệ phát triển, những thiết bị điện tử...",[[Phòng khám FSEC được thiết kế với nhiều màu ...
145,https://vnexpress.net/du-bao-xu-huong-cua-thi-...,"[[Ông Dương Ngọc Báu, CEO Công ty cổ phần Bất ...","[[Dù khó khăn vẫn còn tồn tại, các doanh nghiệ..."


In [124]:
# delete <p class="description"> in description

data_news['description'] = data_news['description'].astype(str)
data_news['description'] = data_news['description'].str.replace('<p class="description">', '')
data_news['description'] = data_news['description'].str.replace('</p>', '')
data_news['description'] = data_news['description'].str.replace('[', '')
data_news['description'] = data_news['description'].str.replace(']', '')

data_news.shape

# data_news.to_excel('data_news_1.xlsx')

C:\Users\VU\AppData\Local\Temp\ipykernel_11568\233120135.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_news['description'] = data_news['description'].str.replace('[', '')
C:\Users\VU\AppData\Local\Temp\ipykernel_11568\233120135.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_news['description'] = data_news['description'].str.replace(']', '')


(147, 3)

In [125]:


# split description 

data_news['description'] = data_news['description'].apply(lambda x: x.split('>'))



In [126]:
# get the last element of description

data_news['description'] = data_news['description'].apply(lambda x: x[-1])



In [127]:
data_news.tail(3)

,link,content,description
144,https://vnexpress.net/loi-the-khi-tham-kham-ma...,"[[Công nghệ phát triển, những thiết bị điện tử...",Phòng khám FSEC được thiết kế với nhiều màu sắ...
145,https://vnexpress.net/du-bao-xu-huong-cua-thi-...,"[[Ông Dương Ngọc Báu, CEO Công ty cổ phần Bất ...","Dù khó khăn vẫn còn tồn tại, các doanh nghiệp ..."
146,https://vnexpress.net/bleu-beaute-dat-muc-tieu...,[[Bleu Beaute Skincare Clinic ra đời vào năm 2...,"Theo Bleu Beaute, đơn vị này mong muốn cung cấ..."


In [128]:
data_news.to_excel('data_news_3.xlsx')

In [167]:
title_news

[<h3 class="title-news"><a data-itm-added="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-1&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-1" data-thumb="1" href="https://vnexpress.net/gia-xang-dau-giam-tiep-ve-muc-thap-nhat-hon-mot-nam-4550952.html" title="Giá xăng, dầu giảm tiếp về mức thấp nhất hơn một năm">Giá xăng, dầu giảm tiếp về mức thấp nhất hơn một năm</a></h3>,
 <h3 class="title-news"><a data-itm-added="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-1&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-1" data-thumb="1" href="https://vnexpress.net/mot-di-khong-tro-lai-4550340.html" title="Một đi không trở lại">Một đi không trở lại</a></h3>,
 <h3 class="title-news"><a data-itm-added="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-4&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-4" data-thumb="1" href="https://vnexpress.net/de-xuat-thoi-gian-luu-tru-voi-khac